<a href="https://colab.research.google.com/github/IANGECHUKI176/deeplearning/blob/main/pytorch/nlp/pytorch-seq2seq/2_Learning_Phrase_Representations_using_RNN_Encoder_Decoder_for_Statistical_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference [Bentrevett Github](https://github.com/bentrevett/pytorch-seq2seq/blob/master/2%20-%20Learning%20Phrase%20Representations%20using%20RNN%20Encoder-Decoder%20for%20Statistical%20Machine%20Translation.ipynb)

Paper: [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/pdf/1406.1078v3.pdf)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from tqdm import tqdm
import random
import math
import time
#pytorch/nlp/pytorch-seq2seq

In [ ]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
import io

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [ ]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            counter.update(tokenizer(string_))
    v2 = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
    v2.set_default_index(0)
    return v2

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

def data_process(filepaths):
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
        de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                                dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                                dtype=torch.long)
        data.append((de_tensor_, en_tensor_))
    return data

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

100%|██████████| 637k/637k [00:00<00:00, 11.2MB/s]
100%|██████████| 569k/569k [00:00<00:00, 11.5MB/s]
100%|██████████| 24.7k/24.7k [00:00<00:00, 7.05MB/s]
100%|██████████| 21.6k/21.6k [00:00<00:00, 2.39MB/s]
100%|██████████| 22.9k/22.9k [00:00<00:00, 7.89MB/s]
100%|██████████| 21.1k/21.1k [00:00<00:00, 36.5MB/s]


In [ ]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)


In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_dim,embed_dim,hidden_dim,dropout):
        super(Encoder,self).__init__()

        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim,embed_dim)
        self.rnn = nn.GRU(embed_dim,hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self,src):
        #src = [seq len,batch_size]
        embedded = self.dropout(self.embedding(src))

        #embedded = [src len,batch size , embed dim]
        outputs,hidden = self.rnn(embedded)
        #outputs =[src len,batch size,hidden dim * n directions]
        #hidden = [n layers * n directions,batch size,hidden dim]

        #outputs are always from the top hidden layer
        return hidden


In [ ]:
class Decoder(nn.Module):
    def __init__(self,output_dim,emb_dim,hid_dim,dropout):
        super(Decoder,self).__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim,emb_dim)
        self.rnn = nn.GRU(emb_dim + hid_dim ,hid_dim)
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2,output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,input,hidden,context):
        #input = [batch size]
        #hidden = [n layers * n directions ,batch size , hidden dim]
        #context = [n layers * n directions ,batch size , hidden dim]

        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden  = [1,batch size , hidden dim]
        #context = [1,batch size , hidden dim]

        input = input.unsqueeze(0)

        #input = [1,batch size]

        embedded = self.dropout(self.embedding(input))
        #[1,batch size,emb dim]

        emb_con = torch.cat([embedded,context],dim = 2)
        # #emb_con = [1,batch_size,emb_dim + hid dim]

        outputs,hidden = self.rnn(emb_con,hidden)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]

        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]

        output = torch.cat([embedded.squeeze(0),hidden.squeeze(0),context.squeeze(0)],dim = 1)
        #output = [batch size, emb dim + hid dim * 2]
        prediction = self.fc_out(output)
        #prediction = [batch size, output dim]
        return prediction,hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self,src,trg,teacher_forcing_ratio = 0.5):
        #src = [seq len ,batch size]
        #trg = [seq len ,batch size]

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        #tensor to store decoded outputs
        outputs = torch.zeros(trg_len,batch_size,trg_vocab_size) .to(device)

        #last hidden state of the encoder is the context
        context = self.encoder(src)

        #context is also used as the initial hidden state of the decoder
        hidden = context

        input = trg[0,:]

        for t in range(1,trg_len):
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output,hidden = self.decoder(input,hidden,context)

            #place predictions in a tensor holding predictions for each token
            outputs[t]  = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        return outputs

In [ ]:
INPUT_DIM =  len(de_vocab)
OUTPUT_DIM = len(en_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5


enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)
model = Seq2Seq(enc,dec).to(device)

In [ ]:
def init_weights(m:nn.Module):
    for name,param in m.named_parameters():
        nn.init.normal_(param.data,mean = 0,std = 0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19215, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10838, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=10838, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model:nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 24,728,918 trainable parameters


In [ ]:
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = en_vocab.get_stoi()['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from tqdm import tqdm

In [ ]:
def train(model:nn.Module,
          iterator : torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          criterion: nn.Module,
          clip: float
          ):
    model.train()
    epoch_loss = 0

    for _ ,(src,trg) in enumerate(tqdm(iterator)):
        src,trg = src.to(device),trg.to(device)

        optimizer.zero_grad()

        output = model(src,trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output = output[1:].view(-1,output.shape[-1])
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output,trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model : nn.Module,
             iterator:torch.utils.data.DataLoader,
             criterion : nn.Module
             ):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for _,(src,trg) in enumerate(tqdm(iterator)):
            src,trg = src.to(device),trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output = output[1:].view(-1,output.shape[-1])
            target = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output,target)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
train_losses = []
val_losses = []
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    train_losses.append(train_loss)
    val_losses.append(valid_loss)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████| 8/8 [00:01<00:00,  5.67it/s]


Epoch: 01 | Time: 1m 18s
	Train Loss: 4.180 | Train PPL:  65.335
	 Val. Loss: 4.820 |  Val. PPL: 124.009


100%|██████████| 8/8 [00:01<00:00,  5.77it/s]


Epoch: 02 | Time: 1m 19s
	Train Loss: 3.857 | Train PPL:  47.311
	 Val. Loss: 4.518 |  Val. PPL:  91.696


100%|██████████| 8/8 [00:01<00:00,  4.97it/s]


...

100%|██████████| 8/8 [00:01<00:00,  5.90it/s]


Epoch: 05 | Time: 1m 18s
	Train Loss: 2.829 | Train PPL:  16.929
	 Val. Loss: 4.056 |  Val. PPL:  57.744


100%|██████████| 8/8 [00:01<00:00,  5.55it/s]


Epoch: 06 | Time: 1m 18s
	Train Loss: 2.535 | Train PPL:  12.612
	 Val. Loss: 3.896 |  Val. PPL:  49.229


 83%|████████▎ | 189/227 [01:03<00:14,  2.60it/s]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.plot(val_losses)

In [ ]:
# Define the translation function
def translate_sentence(model, src_sentence,device, max_len=50):
    model.eval()
    with torch.no_grad():
        if isinstance(src_sentence, str):
            tokens = de_tokenizer(src_sentence)
        else:
            tokens = src_sentence


        tokens = ['<bos>'] + tokens + ['<eos>']
        src_indexes = de_vocab.lookup_indices(tokens)

        src_tensor = torch.tensor(src_indexes).unsqueeze(1).to(device)


        with torch.no_grad():
            context = model.encoder(src_tensor)


        BOS_INDEX = en_vocab.get_stoi()["<bos>"]

        trg_indexes = [BOS_INDEX]

        hidden = context
        for i in range(max_len):

            trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
            with torch.no_grad():
                output, hidden = model.decoder(trg_tensor, hidden, context)


            pred_token = output.argmax(1).item()

            trg_indexes.append(pred_token)

            if pred_token == en_vocab.get_stoi()["<eos>"]:
                break

    trg_tokens = en_vocab.lookup_tokens(trg_indexes)

    return trg_tokens[1:]

In [ ]:
def get_test_data(filepaths): #test_filepaths
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    return raw_de_iter,raw_en_iter

In [ ]:
test_x,test_y = get_test_data(test_filepaths)
src_arr = []
trg_arr = []
for x,y in zip(test_x,test_y):
    src_arr.append(x)
    trg_arr.append(y)

In [ ]:
example_idx = 10
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Mutter und ihr kleiner Sohn genießen einen schönen Tag im Freien.

trg: A mother and her young song enjoying a beautiful day outside.

predicted trg = ['A', 'mother', 'and', 'her', 'daughter', 'enjoy', 'a', 'nice', 'day', 'day', '.', '\n', '<eos>']


In [ ]:
example_idx = 12
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Frau, die in einer Küche eine Schale mit Essen hält.

trg: A woman holding a bowl of food in a kitchen.

predicted trg = ['A', 'woman', 'is', 'in', 'a', 'kitchen', 'kitchen', 'holding', 'a', 'dish', 'of', 'food', '.', '\n', '<eos>']


In [ ]:
example_idx = 14
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Drei Leute sitzen in einer Höhle.

trg: Three people sit in a cave.

predicted trg = ['Three', 'people', 'sit', 'in', 'a', 'wooded', 'area', '.', '\n', '<eos>']


In [ ]:
example_idx = 15
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Ein Mädchen in einem Jeanskleid läuft über einen erhöhten Schwebebalken.

trg: A girl in a jean dress is walking along a raised balance beam.

predicted trg = ['A', 'girl', 'in', 'a', 'dress', 'is', 'walking', 'over', 'a', 'stack', 'of', 'a', '.', '\n', '<eos>']


In [ ]:
example_idx = 16
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Blondine hält mit einem Mann im Sand Händchen.

trg: A blond holding hands with a guy in the sand.

predicted trg = ['A', 'woman', 'holding', 'a', 'man', 'in', 'a', 'man', 'in', 'the', 'sand', '.', '\n', '<eos>']


In [ ]:
example_idx = 17
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Frau in einem Grauen Pulli und mit einer schwarzen Baseballmütze steht in einem Geschäft in der Schlange.

trg: A woman in a gray sweater and black baseball cap is standing in line at a shop.

predicted trg = ['A', 'woman', 'in', 'a', 'black', 'top', 'and', 'black', 'polka', 'dots', 'and', 'black', 'leggings', 'standing', 'in', 'a', 'in', 'a', 'store', '.', '\n', '<eos>']


In [ ]:
example_idx = 19
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Zwei Männer tun so als seien sie Statuen, während Frauen ihnen zusehen.

trg: Two men pretend to be statutes while women look on.

predicted trg = ['Two', 'men', 'are', 'making', 'a', 'picture', 'while', 'while', 'while', 'them', 'watches', 'them', 'watches', 'them', '.', '\n', '<eos>']


In [ ]:
example_idx = 20
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Leute, die vor einem Gebäude stehen.

trg: People standing outside of a building.

predicted trg = ['People', 'standing', 'in', 'front', 'building', 'near', 'a', 'building', '.', '\n', '<eos>']
